In [109]:
import tensorflow as tf
import numpy as np

In [110]:
class EmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self,user_feat_num,user_field_num,
                 ad_feat_num,ad_field_num,
                 context_feat_num,context_field_num,emb_dim,*args,**kwargs):
        super(EmbeddingLayer,self).__init__(*args,**kwargs)
        self.user_feat_num=user_feat_num
        self.user_field_num=user_field_num
        self.ad_feat_num=ad_feat_num
        self.ad_field_num=ad_field_num
        self.context_feat_num=context_feat_num
        self.context_field_num=context_field_num
        self.emb_dim=emb_dim

        self.user_feat_emb_layer=tf.keras.layers.Embedding(input_dim=self.user_feat_num+1,output_dim=self.emb_dim)
        self.ad_feat_emb_layer=tf.keras.layers.Embedding(input_dim=self.ad_feat_num+1,output_dim=self.emb_dim)
        self.context_feat_emb_layer=tf.keras.layers.Embedding(input_dim=self.context_feat_num+1,output_dim=self.emb_dim)

    def call(self, inputs, **kwargs):
        # user_feat_batch: [batch_size, user_field_num]
        # user_behaviors_batch: [batch_size, seq_len, ad_field_dim]
        # ad_feat_batch: [batch_size, ad_field_dim]
        # context_feat_batch: [batch_size, context_field_num]
        user_feat_batch,user_behaviors_batch,ad_feat_batch,context_feat_batch=inputs

        user_emb=self.user_feat_emb_layer(user_feat_batch) # [batch_size, user_field_num, emb_dim]
        user_emb=tf.reshape(user_emb,shape=[-1,self.user_field_num*self.emb_dim]) # [batch_size, user_field_num*emb_dim]

        user_behaviors_emb=self.ad_feat_emb_layer(user_behaviors_batch) # [batch_size, seq_len, ad_field_dim, emb_dim]
        seq_len=user_behaviors_batch.shape[1]
        user_behaviors_emb=tf.reshape(user_behaviors_emb,shape=[-1, seq_len,self.ad_field_num*self.emb_dim]) # [batch_size, seq_len, ad_field_dim*emb_dim]

        ad_emb=self.ad_feat_emb_layer(ad_feat_batch) # [batch_size, ad_field_num, emb_dim]
        ad_emb=tf.reshape(ad_emb,shape=[-1,self.ad_field_num*self.emb_dim]) # [batch_size, ad_field_num*emb_dim]

        context_emb=self.context_feat_emb_layer(context_feat_batch) # [batch_size, context_field_num, emb_dim]
        context_emb=tf.reshape(context_emb,shape=[-1,self.context_field_num*self.emb_dim]) # [batch_size, context_field_num*emb_dim]

        return (user_emb,user_behaviors_emb,ad_emb,context_emb)

user_feat_num=3
user_field_num=3
ad_feat_num=4
ad_field_num=2
context_feat_num=5
context_field_num=3
emb_dim=6
batch_size=10
seq_len=7

user_feat_batch=np.random.randint(1,1+user_feat_num,size=[batch_size,user_field_num]).astype(np.int32)

user_behaviors=[]
for _ in range(batch_size):
    seq=np.random.choice(range(1,1+ad_feat_num),size=[np.random.randint(1,1+seq_len),2]).astype(np.int32).tolist()
    user_behaviors.append(seq)
user_behaviors=tf.keras.preprocessing.sequence.pad_sequences(user_behaviors,maxlen=seq_len,padding="post")

ad_feat_batch=np.random.randint(1,1+ad_feat_num,size=[batch_size,ad_field_num]).astype(np.int32)
context_feat_batch=np.random.randint(1,1+context_feat_num,size=[batch_size,context_field_num]).astype(np.int32)

emb_layer=EmbeddingLayer(user_feat_num=user_feat_num,
user_field_num=user_field_num,
ad_feat_num=ad_feat_num,
ad_field_num=ad_field_num,
context_feat_num=context_feat_num,
context_field_num=context_field_num,emb_dim=emb_dim)

inputs=(user_feat_batch,user_behaviors,ad_feat_batch,context_feat_batch)
user_emb,user_behaviors_emb,ad_emb,context_emb=emb_layer(inputs)
print("user_emb.shape",user_emb.shape)
print("user_behaviors_emb.shape",user_behaviors_emb.shape)
print("ad_emb.shape",ad_emb.shape)
print("context_emb.shape",context_emb.shape)

user_emb.shape (10, 18)
user_behaviors_emb.shape (10, 7, 12)
ad_emb.shape (10, 12)
context_emb.shape (10, 18)


In [111]:
class AttentionLayerCell:
    PRelu="PRelu"
    DICE="DICE"

class InterestLayer(tf.keras.layers.Layer):
    def __init__(self,units_list=None,cell_type=AttentionLayerCell.PRelu,p_relu_alpha=0.2,*args,**kwargs):
        super(InterestLayer,self).__init__(*args,**kwargs)
        units_list=[36] if units_list is None else units_list
        if cell_type == AttentionLayerCell.PRelu:
            self.dense_layers=[tf.keras.layers.Dense(units=units,activation=lambda x:tf.nn.leaky_relu(x,alpha=p_relu_alpha)) for units in units_list]
        else:
            raise ValueError("invalid AttentionLayerCell")
        self.scoring_layer=tf.keras.layers.Dense(units=1,activation=None)

    def call(self, inputs, **kwargs):
        # user_behaviors: [batch_size, seq_len, ad_field_num]
        # user_behaviors_emb: [batch_size, seq_len, emb_dim]
        # ad_emb: [batch_size, emb_dim]
        user_behaviors,user_behaviors_emb,ad_emb=inputs

        # calc attention scores
        # seq_len=user_behaviors.shape[1]
        # emb_dim=ad_emb.shape[-1]
        # ad_emb=tf.reshape(tf.tile(ad_emb,multiples=[1,seq_len]),shape=[-1,seq_len,emb_dim]) #[batch_size, seq_len, emb_dim]
        ad_emb=tf.broadcast_to(tf.expand_dims(ad_emb,axis=1),shape=user_behaviors_emb.shape) #[batch_size, seq_len, emb_dim]
        out_product=tf.multiply(user_behaviors_emb,ad_emb) #[batch_size, seq_len, emb_dim]

        dense_inputs=tf.concat([user_behaviors_emb,out_product,ad_emb],axis=-1) #[batch_size, seq_len, emb_dim*3]
        for dense_layer in self.dense_layers:
            dense_inputs=dense_layer(dense_inputs)

        # dense_inputs: [batch_size, seq_len, units]
        attention_scores=self.scoring_layer(dense_inputs) # [batch_size, seq_len, 1]
        interest_emb=tf.multiply(attention_scores,user_behaviors_emb) # [batch_size, seq_len, emb_dim]

        # mask
        mask=tf.expand_dims(tf.where(tf.not_equal(user_behaviors[:,:,0],0),x=1.,y=0.),axis=-1) # [batch_size, seq_len, 1]
        interest_emb=tf.multiply(interest_emb,mask)
        interest_emb=tf.reduce_sum(interest_emb,axis=1) # [batch_size, emb_dim]
        return interest_emb


inputs=(user_behaviors,user_behaviors_emb,ad_emb)
interest_layer=InterestLayer()
interest_emb = interest_layer(inputs)
print(interest_emb.shape)

(10, 12)


In [112]:

class DIN(tf.keras.Model):
    def __init__(self,user_feat_num,user_field_num,
                 ad_feat_num,ad_field_num,
                 context_feat_num,context_field_num,emb_dim,
                 dense_units_list=None,dense_cell_type=AttentionLayerCell.PRelu,dense_p_relu_alpha=0.2,
                 attention_units_list=None,attention_cell_type=AttentionLayerCell.PRelu,attention_p_relu_alpha=0.2,
                 *args,**kwargs):
        super(DIN,self).__init__(*args,**kwargs)
        self.emb_layer=EmbeddingLayer(user_feat_num=user_feat_num,user_field_num=user_field_num,
                                      ad_feat_num=ad_feat_num,ad_field_num=ad_field_num,
                                      context_feat_num=context_feat_num,context_field_num=context_field_num,
                                      emb_dim=emb_dim)
        self.interest_layer=InterestLayer(units_list=attention_units_list,cell_type=attention_cell_type,
                                          p_relu_alpha=attention_p_relu_alpha)
        dense_units_list=[200,80] if dense_units_list is None else dense_units_list
        if dense_cell_type == AttentionLayerCell.PRelu:
            self.dense_layers=[tf.keras.layers.Dense(units=units,activation=lambda x:tf.nn.leaky_relu(x,alpha=dense_p_relu_alpha)) for units in dense_units_list]
        else:
            raise ValueError("invalid AttentionLayerCell")
        self.scoring_layer=tf.keras.layers.Dense(units=2,activation=None)

    def call(self, inputs, training=None, mask=None):
        _,user_behaviors,_,_=inputs
        user_emb,user_behaviors_emb,ad_emb,context_emb=self.emb_layer(inputs)
        interest_emb=self.interest_layer((user_behaviors,user_behaviors_emb,ad_emb))

        dense_inputs=tf.concat([user_emb,interest_emb,context_emb],axis=1)
        for dense_layer in self.dense_layers:
            dense_inputs=dense_layer(dense_inputs)
        output=self.scoring_layer(dense_inputs)

        return output

user_feat_num=3
user_field_num=3
ad_feat_num=4
ad_field_num=2
context_feat_num=5
context_field_num=3
emb_dim=6
batch_size=10
seq_len=7
user_feat_batch=np.random.randint(1,1+user_feat_num,size=[batch_size,user_field_num]).astype(np.int32)

user_behaviors=[]
for _ in range(batch_size):
    seq=np.random.choice(range(1,1+ad_feat_num),size=[np.random.randint(1,1+seq_len),2]).astype(np.int32).tolist()
    user_behaviors.append(seq)
user_behaviors=tf.keras.preprocessing.sequence.pad_sequences(user_behaviors,maxlen=seq_len,padding="post")

ad_feat_batch=np.random.randint(1,1+ad_feat_num,size=[batch_size,ad_field_num]).astype(np.int32)
context_feat_batch=np.random.randint(1,1+context_feat_num,size=[batch_size,context_field_num]).astype(np.int32)

din=DIN(user_feat_num=user_feat_num,
user_field_num=user_field_num,
ad_feat_num=ad_feat_num,
ad_field_num=ad_field_num,
context_feat_num=context_feat_num,
context_field_num=context_field_num,emb_dim=emb_dim)

inputs=(user_feat_batch,user_behaviors,ad_feat_batch,context_feat_batch)
output=din(inputs)
print(output.shape)

(10, 2)
